In [8]:
#if geckodriver in local directory, this will add to path temporarily
import os
path = os.getcwd()
os.environ["PATH"] += os.pathsep + path

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import trange
import pickle
import time
import sys

In [5]:
main_url = 'https://www.tripadvisor.com/Airlines'

In [6]:
def get_links_to_scrape(main_url, start=0, num_pages=10):
    '''
    navigates to airline page, extracts html links to airlines for future scrape.
    '''
    driver = webdriver.Firefox()
    driver.get(main_url)
    driver.implicitly_wait(100)

    for n in trange(start):
        button = driver.find_element_by_class_name("nav.next")
        button.click()
        time.sleep(2)

    url_list = []
    for n in trange(num_pages):
        urls = driver.find_elements_by_class_name('detailsLink')
        for url in urls[:10]:
            href = url.get_attribute('href')
            url_list.append(href)

        button = driver.find_element_by_class_name("nav.next")
        button.click()
        time.sleep(2)

    driver.quit()
    new_airlines = '\n'.join(url_list)
    with open('airlines.txt', 'a') as outfile:
        outfile.write(new_airlines)

    return url_list

In [3]:
#list of airlines (also output to screen)
get_links_to_scrape(main_url, start=0, num_pages=1)

In [10]:
def start_scrape(url, start=0):
    '''
    starts selenium web driver
    param: url - the page url of the page you wish to scrape
    param = int: if scrape crashes - change parameter (start) to page number where crash happened to restart from
    that point

    '''
    driver = webdriver.Firefox()
    driver.get(url)
    driver.implicitly_wait(100)

    for n in trange(start):
        button = driver.find_element_by_link_text("Next")
        button.click()
    return driver

In [14]:
def trip_advisor_scrape(driver, url, start=0, num_reviews=1500):
    basename = url.split("Reviews-")[-1]
    soup_list = []
    for n in trange(num_reviews):
        try:
            # expand the 'read more' element
            elements = driver.find_element_by_class_name('_36B4Vw6t')
            elements.click()
        except:
            pass
        soup = BeautifulSoup(driver.page_source, 'html.parser')
# this class identifier may be changed in future
        review_soup = soup.find_all('div', attrs={'class': 'oETBfkHU'})
        soup_list.append(str(review_soup))
        try:
            button = driver.find_element_by_link_text("Next")
            button.click()
            time.sleep(1)
        except:
            sys.setrecursionlimit(10000)
            pickle.dump(soup_list, open(f'{basename}_{start}-{start + n}.pkl', 'wb'))
            break

    print('finished at: ', n+1)
    sys.setrecursionlimit(10000)
    #this will dump to local directory
    pickle.dump(soup_list, open(f'{basename}_{start}-{start + n}.pkl', 'wb'))

In [2]:
# uncomment one of below for either mass scrape or select airlines
# urls = ['https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-Adria-Airways-No-Longer-Operating']
# urls = get_links_to_scrape(main_url)

for url in urls:
    start = 0
    num_reviews = 15
    driver = start_scrape(url, start=int(start))
    trip_advisor_scrape(driver, url, start=int(start), num_reviews=int(num_reviews))
    driver.quit()